In [19]:
import numpy as np
import pandas as pd
from ase.io import read
import matplotlib.pyplot as plt
from funciones import get_molecules
from funciones import get_data

# Reading the original file, with all the frames of relaxation

In [2]:
frames = read("pw1.out", index=":")
len(frames)

687

In [3]:
frames[0][0].symbol

'Cl'

# Getting the indeces of the molecules in the file.

The indeces never change, so it is only needed to get the indeces of one frame

In [4]:
sol = get_molecules(["alcl3_opt.xyz","1-Ethyl-3-methylimidazolium_opt.xyz"], "pw1.out")
len(sol) # number of molecules in the file "pw1.out"

16

# Getting the center of masses of every molecule in every frame

In [5]:
frames_pos_data = np.empty(len(frames), dtype=list)
for i in range(len(frames)): # loop for frames in the quantum espresso file
	center_of_masses = []
	for j in range(len(sol)):
		center_of_masses.append(frames[i].get_center_of_mass(scaled=False,indices= sol[j].idx_list)) # Collecting the center of masses of every molecule in the i frame

	frames_pos_data[i] = center_of_masses # Once it's finished the loop for molecules, the variable "center_of_masses" has the center of mass of every molecule in the i frame, so 
										# it will be storaged inside the list "frames_pos_data" in the corresponding index

## Example: How to get the center of masses of the molecules in frame cero

In [6]:
frames_pos_data[0]

[array([ 4.16288287, 10.18861236,  2.0208343 ]),
 array([10.18575708, 13.03608404,  3.56249876]),
 array([16.07289281,  9.71448288,  2.83325217]),
 array([5.35341414, 9.29302549, 3.66774788]),
 array([14.43985411, 11.59458826,  2.79307977]),
 array([15.29117082,  4.16074363,  5.35666806]),
 array([7.93871678, 6.56344432, 3.53328788]),
 array([ 8.20381214, 13.64510408,  5.52380239]),
 array([13.80254545,  2.86025862,  1.94964902]),
 array([8.40000395, 1.29230339, 4.23040053]),
 array([13.93671922, 15.27663519,  1.65429853]),
 array([ 3.69665049, 14.49873854,  1.55326268]),
 array([ 1.32362808, 10.5407118 ,  4.0242334 ]),
 array([15.39184352, 13.11914078,  5.51226666]),
 array([13.53454489,  7.0270404 ,  3.54951353]),
 array([3.49433959, 5.08250682, 5.48393925])]

# The center of mass of the mixture in the final frame

In [23]:
all_center_of_mass = frames[-1].get_center_of_mass(scaled=False)
all_center_of_mass

array([9.76520161, 9.24159229, 3.56338806])

## Moving the reference from the cell axis to the axis in the center of mass of the final configuration of the mixture

In [8]:
frames_pos_data_T = np.empty(len(frames), dtype=list)
for i in range(len(frames)):
	frames_pos_data_T[i] = frames_pos_data[i] - all_center_of_mass # Now the origin is moved to the center of mass of the mixture in the final configuration.

In [9]:
len(frames_pos_data_T)

687

# Calculating the radial distance from the center of mass of the mixture to the each molecule

In [10]:
frames_radius_distance = np.empty(len(frames), dtype=list)
for i in range(len(frames)): # loop for every frame
	aux = np.empty(len(sol),dtype=float)
	for j in range(len(sol)): # loop for every molecule
		aux[j] = np.sqrt(frames_pos_data_T[i][j].dot(frames_pos_data_T[i][j])) # Collecting the radius distance of molecule j in frame i
	
	frames_radius_distance[i] = aux # Collecting the radius distance of all the molecules in frame i

In [11]:
frames_radius_distance = np.vstack(frames_radius_distance, dtype=float)
frames_radius_distance

array([[5.88746927, 3.81772646, 6.36739288, ..., 7.10581517, 4.37177096,
        7.76596545],
       [5.88802367, 3.81846085, 6.36582067, ..., 7.10736933, 4.37085725,
        7.76657525],
       [5.88889596, 3.81909293, 6.36392965, ..., 7.10902589, 4.37006356,
        7.76721626],
       ...,
       [5.46694594, 5.21198885, 6.34149469, ..., 8.26912716, 3.76125357,
        9.70705275],
       [5.46862506, 5.2141362 , 6.34236002, ..., 8.26630159, 3.76191347,
        9.71135159],
       [5.47028388, 5.21626167, 6.34319028, ..., 8.26347048, 3.76259828,
        9.71567377]], shape=(687, 16))

In [12]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.patches import Patch 

# Crear figura
fig, ax = plt.subplots(figsize=(12, 8))

labels = [f"Molecule {i+1}" for i in range(len(frames_radius_distance[0]))]

color_list = ['slategrey','forestgreen']

# Crear lista de colores personalizada
colors = [f"{color_list[0]}" if i < 7 else f"{color_list[1]}" for i in range(len(labels))]

x = np.arange(len(labels))
bars = ax.bar(x, frames_radius_distance[0], tick_label=labels, color = colors)

plt.setp(ax.get_xticklabels(), rotation=90)
ax.set_ylim(0, np.max(frames_radius_distance) * 1.1)
ax.set_ylabel("Radial Distance from the final CM of the mixture to each molecule")

# Leyenda manual
legend_elements = [
    Patch(facecolor=f"{color_list[0]}", label=r'$AlCl_3$'),
    Patch(facecolor=f"{color_list[1]}", label='1-Ethyl-3-methylimidazolium')
]
ax.legend(handles=legend_elements, loc='upper right')

# Función de actualización
def update(frame):
    y_vals = frames_radius_distance[frame]
    for bar, y in zip(bars, y_vals):
        bar.set_height(y)
    ax.set_title(f"Frame {frame}", pad =10)
    return bars

plt.tight_layout()
plt.subplots_adjust(top=0.88)
# Crear animación
ani = animation.FuncAnimation(fig, update, frames=frames_radius_distance.shape[0], interval=50, blit=False)

# Guardar el video
Writer = animation.writers['ffmpeg']
writer = Writer(fps=10, metadata=dict(artist='Me'), bitrate=1800)
ani.save("animacion.mp4", writer=writer)

plt.close(fig)  # Importante si no querés mostrar la figura estática

#HTML(anim.to_jshtml())

In [41]:
aux = get_data(["alcl3_opt.xyz","1-Ethyl-3-methylimidazolium_opt.xyz"], "pw1.out")


cols = aux.columns

# Iteramos en pasos de 3 (x,y,z)
for i in range(0, len(cols), 3):
    # Columnas de x,y,z para esa molécula
    x_col = cols[i]
    y_col = cols[i+1]
    z_col = cols[i+2]

    # Sumamos el desplazamiento
    aux[x_col] = aux[x_col] - all_center_of_mass[0]
    aux[y_col] = aux[y_col] - all_center_of_mass[1]
    aux[z_col] = aux[z_col] - all_center_of_mass[2]

aux

,Cl_1_x,Cl_1_y,Cl_1_z,Cl_2_x,Cl_2_y,Cl_2_z,Cl_3_x,Cl_3_y,Cl_3_z,Al_4_x,...,NNCCCCCCHHHHHHHHHHH_13_z,NNCCCCCCHHHHHHHHHHH_14_x,NNCCCCCCHHHHHHHHHHH_14_y,NNCCCCCCHHHHHHHHHHH_14_z,NNCCCCCCHHHHHHHHHHH_15_x,NNCCCCCCHHHHHHHHHHH_15_y,NNCCCCCCHHHHHHHHHHH_15_z,NNCCCCCCHHHHHHHHHHH_16_x,NNCCCCCCHHHHHHHHHHH_16_y,NNCCCCCCHHHHHHHHHHH_16_z
0,-7.246442,-0.038900,-2.643836,-6.054064,2.243826,0.190137,-3.506450,0.636134,-2.173963,-5.602318,...,0.460845,5.626642,3.877548,1.948879,3.769343,-2.214552,-0.013875,-6.270862,-4.159085,1.920551
1,-7.244801,-0.037709,-2.646501,-6.055643,2.243782,0.190205,-3.506193,0.636474,-2.176582,-5.604733,...,0.461074,5.627377,3.878171,1.951183,3.768633,-2.213959,-0.013684,-6.271113,-4.159710,1.920847
2,-7.243000,-0.036407,-2.649549,-6.057231,2.244549,0.189534,-3.506339,0.636561,-2.180234,-5.607418,...,0.461267,5.628080,3.878780,1.953977,3.767920,-2.213606,-0.013498,-6.271241,-4.160513,1.921282
3,-7.241040,-0.034995,-2.652978,-6.058832,2.246121,0.188140,-3.506888,0.636391,-2.184916,-5.610366,...,0.461425,5.628755,3.879376,1.957257,3.767205,-2.213492,-0.013312,-6.271244,-4.161498,1.921853
4,-7.238922,-0.033470,-2.656794,-6.060450,2.248494,0.186036,-3.507837,0.635964,-2.190629,-5.613573,...,0.461552,5.629407,3.879962,1.961020,3.766488,-2.213613,-0.013124,-6.271122,-4.162668,1.922561
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
682,-6.838287,-0.465849,-2.590289,-4.248003,0.004387,-0.925726,-3.446992,-1.294163,-3.562554,-5.108694,...,1.360865,6.431884,5.079761,1.139539,3.331015,-1.690519,0.429099,-6.076270,-7.027329,2.785154
683,-6.832543,-0.464754,-2.593525,-4.253494,0.005863,-0.930948,-3.453977,-1.300021,-3.554490,-5.108361,...,1.360921,6.430268,5.077961,1.136253,3.332290,-1.689131,0.430027,-6.078168,-7.030553,2.787672
684,-6.826827,-0.463704,-2.596851,-4.258968,0.007464,-0.936022,-3.460988,-1.305944,-3.546333,-5.107938,...,1.360988,6.428619,5.076190,1.132968,3.333604,-1.687723,0.430934,-6.080101,-7.033779,2.790196
685,-6.821144,-0.462697,-2.600267,-4.264422,0.009188,-0.940941,-3.468020,-1.311931,-3.538099,-5.107428,...,1.361065,6.426938,5.074448,1.129685,3.334957,-1.686293,0.431822,-6.082069,-7.037008,2.792725


In [42]:
df = aux.iloc[:,-16*3:]
df

,ClClClAl_1_x,ClClClAl_1_y,ClClClAl_1_z,ClClClAl_2_x,ClClClAl_2_y,ClClClAl_2_z,ClClClAl_3_x,ClClClAl_3_y,ClClClAl_3_z,ClClClAl_4_x,...,NNCCCCCCHHHHHHHHHHH_13_z,NNCCCCCCHHHHHHHHHHH_14_x,NNCCCCCCHHHHHHHHHHH_14_y,NNCCCCCCHHHHHHHHHHH_14_z,NNCCCCCCHHHHHHHHHHH_15_x,NNCCCCCCHHHHHHHHHHH_15_y,NNCCCCCCHHHHHHHHHHH_15_z,NNCCCCCCHHHHHHHHHHH_16_x,NNCCCCCCHHHHHHHHHHH_16_y,NNCCCCCCHHHHHHHHHHH_16_z
0,-5.602319,0.947020,-1.542554,0.420555,3.794492,-0.000889,6.307691,0.472891,-0.730136,-4.411787,...,0.460845,5.626642,3.877548,1.948879,3.769343,-2.214552,-0.013875,-6.270862,-4.159085,1.920551
1,-5.602723,0.946932,-1.543257,0.418799,3.795425,-0.000476,6.306260,0.473043,-0.728693,-4.411044,...,0.461074,5.627377,3.878171,1.951183,3.768633,-2.213959,-0.013684,-6.271113,-4.159710,1.920847
2,-5.603248,0.947007,-1.544631,0.416890,3.796271,-0.000373,6.304541,0.473109,-0.727005,-4.410138,...,0.461267,5.628080,3.878780,1.953977,3.767920,-2.213606,-0.013498,-6.271241,-4.160513,1.921282
3,-5.603895,0.947241,-1.546672,0.414835,3.797027,-0.000572,6.302542,0.473085,-0.725077,-4.409073,...,0.461425,5.628755,3.879376,1.957257,3.767205,-2.213492,-0.013312,-6.271244,-4.161498,1.921853
4,-5.604663,0.947633,-1.549376,0.412633,3.797694,-0.001066,6.300272,0.472971,-0.722911,-4.407846,...,0.461552,5.629407,3.879962,1.961020,3.766488,-2.213613,-0.013124,-6.271122,-4.162668,1.922561
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
682,-4.897906,-0.803922,-2.283485,-0.097343,5.197879,-0.303280,6.251183,1.051492,-0.091313,-3.595406,...,1.360865,6.431884,5.079761,1.139539,3.331015,-1.690519,0.429099,-6.076270,-7.027329,2.785154
683,-4.899628,-0.806034,-2.283169,-0.098944,5.199991,-0.304175,6.252176,1.051177,-0.091955,-3.594499,...,1.360921,6.430268,5.077961,1.136253,3.332290,-1.689131,0.430027,-6.078168,-7.030553,2.787672
684,-4.901342,-0.808083,-2.282838,-0.100556,5.202082,-0.305049,6.253121,1.050936,-0.092588,-3.593632,...,1.360988,6.428619,5.076190,1.132968,3.333604,-1.687723,0.430934,-6.080101,-7.033779,2.790196
685,-4.903048,-0.810067,-2.282493,-0.102180,5.204152,-0.305900,6.254017,1.050769,-0.093213,-3.592804,...,1.361065,6.426938,5.074448,1.129685,3.334957,-1.686293,0.431822,-6.082069,-7.037008,2.792725


In [43]:
# Obtener nombres únicos de las moléculas sin el sufijo _x, _y, _z
base_names = sorted(set(col.rsplit('_', 1)[0] for col in df.columns if col.endswith('_x')))

# Para cada molécula, calcular su distancia radial y agregarla al DataFrame
for name in base_names:
    x = df[f"{name}_x"]
    y = df[f"{name}_y"]
    z = df[f"{name}_z"]
    df[f"{name}_r"] = np.sqrt(x**2 + y**2 + z**2)

In [44]:
DF = df.iloc[:,-16:]

DF

,ClClClAl_1_r,ClClClAl_2_r,ClClClAl_3_r,ClClClAl_4_r,ClClClAl_5_r,ClClClAl_6_r,ClClClAl_7_r,NNCCCCCCHHHHHHHHHHH_10_r,NNCCCCCCHHHHHHHHHHH_11_r,NNCCCCCCHHHHHHHHHHH_12_r,NNCCCCCCHHHHHHHHHHH_13_r,NNCCCCCCHHHHHHHHHHH_14_r,NNCCCCCCHHHHHHHHHHH_15_r,NNCCCCCCHHHHHHHHHHH_16_r,NNCCCCCCHHHHHHHHHHH_8_r,NNCCCCCCHHHHHHHHHHH_9_r
0,5.887469,3.817726,6.367393,4.413321,5.289834,7.717980,3.241825,8.093199,7.580760,8.276805,8.553377,7.105815,4.371771,7.765965,5.066762,7.721769
1,5.888024,3.818461,6.365821,4.412533,5.291248,7.718613,3.241610,8.091508,7.578197,8.276048,8.553432,7.107369,4.370857,7.766575,5.065641,7.721251
2,5.888896,3.819093,6.363930,4.411592,5.292877,7.719171,3.241555,8.089760,7.575625,8.275426,8.552995,7.109026,4.370064,7.767216,5.064847,7.720631
3,5.890085,3.819621,6.361728,4.410500,5.294713,7.719653,3.241659,8.087958,7.573059,8.274944,8.552075,7.110787,4.369389,7.767888,5.064384,7.719917
4,5.891589,3.820045,6.359224,4.409254,5.296750,7.720061,3.241922,8.086102,7.570508,8.274602,8.550684,7.112660,4.368831,7.768592,5.064255,7.719122
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
682,5.463522,5.207629,6.339658,4.039764,4.155532,8.541705,3.890105,7.686137,8.702888,9.394787,8.249229,8.274760,3.760005,9.698530,6.186627,7.713965
683,5.465245,5.209820,6.340594,4.040327,4.158365,8.545519,3.890258,7.684702,8.700522,9.397550,8.249707,8.271947,3.760618,9.702779,6.186777,7.713594
684,5.466946,5.211989,6.341495,4.040926,4.161184,8.549351,3.890440,7.683236,8.698171,9.400334,8.250150,8.269127,3.761254,9.707053,6.186879,7.713242
685,5.468625,5.214136,6.342360,4.041562,4.163988,8.553200,3.890649,7.681738,8.695836,9.403137,8.250556,8.266302,3.761913,9.711352,6.186934,7.712911


In [47]:
DF.describe()

,ClClClAl_1_r,ClClClAl_2_r,ClClClAl_3_r,ClClClAl_4_r,ClClClAl_5_r,ClClClAl_6_r,ClClClAl_7_r,NNCCCCCCHHHHHHHHHHH_10_r,NNCCCCCCHHHHHHHHHHH_11_r,NNCCCCCCHHHHHHHHHHH_12_r,NNCCCCCCHHHHHHHHHHH_13_r,NNCCCCCCHHHHHHHHHHH_14_r,NNCCCCCCHHHHHHHHHHH_15_r,NNCCCCCCHHHHHHHHHHH_16_r,NNCCCCCCHHHHHHHHHHH_8_r,NNCCCCCCHHHHHHHHHHH_9_r
count,687.000000,687.000000,687.000000,687.000000,687.000000,687.000000,687.000000,687.000000,687.000000,687.000000,687.000000,687.000000,687.000000,687.000000,687.000000,687.000000
mean,5.533322,4.561080,6.334417,4.185481,4.634656,8.128150,3.741209,7.849427,8.515111,8.938772,8.324988,7.807496,4.072826,8.924976,5.707379,7.923101
std,0.304254,0.465804,0.099635,0.119542,0.457325,0.226792,0.222552,0.167543,0.534114,0.317632,0.156809,0.472373,0.249304,0.665044,0.287784,0.153138
min,5.211771,3.793671,6.177547,3.997408,4.054853,7.717980,3.241555,7.623920,7.550646,8.274366,8.061023,7.105815,3.746675,7.765965,5.064255,7.706190
25%,5.291627,4.100427,6.265938,4.041049,4.212071,7.932413,3.584037,7.682110,7.957906,8.717751,8.174609,7.297431,3.796560,8.295545,5.569043,7.775718
50%,5.368631,4.805807,6.312475,4.205611,4.546995,8.217847,3.844511,7.866014,8.792806,9.032957,8.387604,7.850636,4.091820,9.188310,5.724219,7.907339
75%,5.883314,4.929512,6.424488,4.287532,5.105330,8.299453,3.925761,8.026405,8.920986,9.200324,8.449187,8.299095,4.321355,9.517328,5.896788,8.070942
max,6.057056,5.216262,6.509728,4.413321,5.350238,8.557066,3.942501,8.093199,9.106481,9.405960,8.553432,8.427808,4.407485,9.715674,6.186942,8.157319
